# Intro To TDR (CRISPI) Workshop Set-Up

This notebook runs through the following steps in **TDR Prod**:
- Authenticate using B2C
- Using an existing user ed billing profile, create a *dataset* based off of BTRIS pilot data
- Ingest data into the *dataset*

Prerequisites to run:
- You must supply your own gcloud auth token for a user that has access to the user ed billing profile.
- You must supply the index for use in generating a unique dataset name -- this will serve as its suffix.

Future opportunities:
- Support for registering participants, which would include adding them to the Firecloud group and adding as a dataset SNAPSHOT_CREATOR on creation.
- Support for bulk setup: supply several emails at once.
- Why does importing tdr_utils not work?
- Why do gcloud auth commands not work?
- Rewrite in dsp_scripts with Poetry and Docker
- Script workshop cleanup (it's easier because we've tagged these datasets)

In [ ]:
## User Ed billing profile
billing_profile_id = "50788d67-d7a9-4a77-ab07-1d5d2530dfb9"

## Set up index for dataset naming
idx = 30
token = "<your token here>"

## Import dependencies

In [ ]:
import json, uuid, time
from IPython.core.display import display, clear_output, HTML
from data_repo_client import ApiException, JobsApi

# Helper class with methods to make the Terra Data Repository easier to use via Python
class TdrUtils:

    def __init__(self, jobs_api):
        self.jobs_api = jobs_api

    # Format and print TDR objects
    def tdr_object_to_json(self, obj):
        if (hasattr(obj, "to_dict")):
            return json.dumps(obj.to_dict(), indent=4)
        else:
            return json.dumps(obj, indent=4)


    # Format TDR objects
    def pretty_print_tdr_object(self, obj):
        print(self.tdr_object_to_json(obj))

    # Wait for an monitoy the status of a job
    def wait_for_job(self, job_model):
        counter = 0
        max_dots = 8
        result = job_model
        job_str = '"%s" (%s)' % (job_model.description, job_model.id)
        while True:
            counter += 1
            clear_output(wait=True)
            if (result == None or result.job_status == "running"):
                num_dots = counter % max_dots
                dots = '.' * (num_dots)
                display(HTML('<b>Running job %s %s</b>' %(job_str, dots)))
                time.sleep(3)
                result = self.jobs_api.retrieve_job(job_model.id)
            elif (result.job_status == 'failed'):
                display(HTML('<b>Running job %s: <span style="color:rgb(219, 50, 20);">failed</span></b>' % job_str))
                try:
                    result = self.jobs_api.retrieve_job_result(job_model.id)
                except ApiException as e:
                    result = display(HTML('<p>Job result is:</p><pre>%s</pre>' % self.tdr_object_to_json(e)))
                return result
            elif (result.job_status == "succeeded"):
                display(HTML('<b>Running job %s: <span style="color:rgb(116, 174, 6);">succeeded</span></b>' % job_str))
                result = self.jobs_api.retrieve_job_result(job_model.id)
                display(HTML('<p>Job result is:</p><hr/><pre>%s</pre>' % self.tdr_object_to_json(result)))
                return result
            else:
                raise "Unrecognized job state %s" % result.job_status


    # Convert bytes representation of UUID into string representation
    def UUID(self, bytes):
        return str(uuid.UUID(bytes=bytes))

In [ ]:
%%capture
import sys

%load_ext autoreload
%autoreload 2

import datetime, uuid, urllib, os, time
## TODO: why does this import not work?
## from tdr_utils import TdrUtils
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
from data_repo_client import ApiClient, ApiException, Configuration, DatasetsApi, SnapshotsApi, JobsApi, ResourcesApi, DataRepositoryServiceApi


## Set configuration

In [ ]:
# Set up configuration
config = Configuration()

config.host="https://data.terra.bio"

# TODO: why does this token fetch not work?
# token=!gcloud auth print-access-token okotsopo.broad.test@gmail.com

config.access_token = token  #token[0]
api_client = ApiClient(configuration=config)
api_client.client_side_validation = False

# Create required API Clients
jobs_api = JobsApi(api_client=api_client)
resources_api = ResourcesApi(api_client=api_client)
datasets_api = DatasetsApi(api_client=api_client)
snapshots_api = SnapshotsApi(api_client=api_client)
tdr_utils = TdrUtils(jobs_api)

## Create Dataset
This process will trigger the creation of a dataset based off of BTRIS pilot data.
Tables:
- demographics
- lab_event
- lab_obs

In [ ]:
dataset_request = {
  "name": f"BTRIS_CRISPI_TDR_Workshop_{idx}",
  "description": "This is a dataset for the Feb 28, 2024 CRISPI TDR workshop.  It is a subset of the [BTRIS pilot dataset's schema](https://btris.nih.gov/index.html).",
  "cloudPlatform": "gcp",
  "defaultProfileId": "50788d67-d7a9-4a77-ab07-1d5d2530dfb9",
  "region": "us-central1",
  "schema": {
    "tables": [
      {
        "name": "demographics",
        "columns": [
          {
            "name": "MRN",
            "datatype": "string",
            "required": True
          },
          {
            "name": "First_Name",
            "datatype": "string"
          },
          {
            "name": "Last_Name",
            "datatype": "string"
          },
          {
            "name": "Middle_Name",
            "datatype": "string"
          },
          {
            "name": "Birth_Date",
            "datatype": "datetime"
          },
          {
            "name": "Death_Date",
            "datatype": "datetime"
          },
          {
            "name": "Race",
            "datatype": "string"
          },
          {
            "name": "Race_Concept",
            "datatype": "string"
          },
          {
            "name": "Ethnic_Group",
            "datatype": "string"
          },
          {
            "name": "Ethnic_Group_Concept",
            "datatype": "string"
          },
          {
            "name": "Gender",
            "datatype": "string"
          },
          {
            "name": "Gender_Concept",
            "datatype": "string"
          },
          {
            "name": "Maritial_Status",
            "datatype": "string"
          },
          {
            "name": "Maritial_Status_Concept",
            "datatype": "string"
          },
          {
            "name": "Religion",
            "datatype": "string"
          },
          {
            "name": "Religion_Concept",
            "datatype": "string"
          },
          {
            "name": "Occupation",
            "datatype": "string"
          },
          {
            "name": "Occupation_Concept",
            "datatype": "string"
          },
          {
            "name": "Language",
            "datatype": "string"
          },
          {
            "name": "Language_Concept",
            "datatype": "string"
          },
          {
            "name": "Phone_number",
            "datatype": "string"
          },
          {
            "name": "Address_Line1",
            "datatype": "string"
          },
          {
            "name": "Addresss_City",
            "datatype": "string"
          },
          {
            "name": "Address_State",
            "datatype": "string"
          },
          {
            "name": "Address_Country",
            "datatype": "string"
          },
          {
            "name": "Birth_City",
            "datatype": "string"
          },
          {
            "name": "Birth_State",
            "datatype": "string"
          },
          {
            "name": "Birth_Country",
            "datatype": "string"
          },
          {
            "name": "Data_Source",
            "datatype": "string"
          },
          {
            "name": "Blood_Type",
            "datatype": "string"
          },
          {
            "name": "Rh_Factor",
            "datatype": "string"
          },
          {
            "name": "Date_Created",
            "datatype": "datetime"
          },
          {
            "name": "Date_Modified",
            "datatype": "datetime"
          },
          {
            "name": "area_code",
            "datatype": "string"
          }
        ]
      },
      {
        "name": "lab_event",
        "columns": [
          {
            "name": "MRN",
            "datatype": "string",
            "required": True
          },
          {
            "name": "Event_GUID",
            "datatype": "string",
            "required": True
          },
          {
            "name": "Child_Flag",
            "datatype": "boolean"
          },
          {
            "name": "Parent_GUID",
            "datatype": "string"
          },
          {
            "name": "Event_Name",
            "datatype": "string"
          },
          {
            "name": "Event_Name_CONCEPT",
            "datatype": "string"
          },
          {
            "name": "Primary_Date_Time",
            "datatype": "datetime"
          },
          {
            "name": "Event_ID",
            "datatype": "string"
          },
          {
            "name": "Sequence",
            "datatype": "string"
          },
          {
            "name": "Appl_Source_CD",
            "datatype": "string"
          },
          {
            "name": "Blinded_Flag",
            "datatype": "boolean"
          },
          {
            "name": "Status",
            "datatype": "string"
          },
          {
            "name": "Status_Concept",
            "datatype": "string"
          },
          {
            "name": "Date_created",
            "datatype": "datetime"
          },
          {
            "name": "date_modified",
            "datatype": "datetime"
          },
          {
            "name": "Domain_Concept",
            "datatype": "string"
          }
        ]
      },
      {
        "name": "lab_obs",
        "columns": [
          {
            "name": "Observation_GUID",
            "datatype": "string",
            "required": True
          },
          {
            "name": "Event_GUID",
            "datatype": "string",
            "required": True
          },
          {
            "name": "MRN",
            "datatype": "string",
            "required": True
          },
          {
            "name": "Child_Flag",
            "datatype": "boolean"
          },
          {
            "name": "Parent_GUID",
            "datatype": "string"
          },
          {
            "name": "Observation_Name",
            "datatype": "string"
          },
          {
            "name": "Observation_Name_CONCEPT",
            "datatype": "string"
          },
          {
            "name": "Primary_Date_Time",
            "datatype": "datetime"
          },
          {
            "name": "Observation_Value_Text",
            "datatype": "string"
          },
          {
            "name": "Observation_Value_Numeric",
            "datatype": "float"
          },
          {
            "name": "Observation_Value_CONCEPT",
            "datatype": "string"
          },
          {
            "name": "Observation_Value_Name",
            "datatype": "string"
          },
          {
            "name": "Sequence",
            "datatype": "float"
          },
          {
            "name": "Unit_of_Measure",
            "datatype": "string"
          },
          {
            "name": "c_Range",
            "datatype": "string"
          },
          {
            "name": "Blinded_Flag",
            "datatype": "boolean"
          },
          {
            "name": "Appl_Source_CD",
            "datatype": "string"
          },
          {
            "name": "Status",
            "datatype": "string"
          },
          {
            "name": "Status_Concept",
            "datatype": "string"
          },
          {
            "name": "Date_Created",
            "datatype": "datetime"
          },
          {
            "name": "Domain_Concept",
            "datatype": "string"
          },
          {
            "name": "date_modified",
            "datatype": "datetime"
          }
        ]
      }
    ],
    "relationships": [
      {
        "name": "demographics__to__lab_event",
        "from": {
          "table": "demographics",
          "column": "MRN"
        },
        "to": {
          "table": "lab_event",
          "column": "MRN"
        }
      },
      {
        "name": "lab_event__to__lab_obs",
        "from": {
          "table": "lab_event",
          "column": "Event_GUID"
        },
        "to": {
          "table": "lab_obs",
          "column": "Event_GUID"
        }
      }
    ],
    "assets": [
      {
        "name": "Lab_Event_And_Obs_With_Demographics",
        "tables": [
          {
            "name": "lab_event",
            "columns": [
              "MRN",
              "Event_GUID",
              "Event_Name",
              "Event_Name_CONCEPT",
              "Primary_Date_Time",
              "Event_ID"
            ]
          },
          {
            "name": "lab_obs",
            "columns": [
              "MRN",
              "Event_GUID",
              "Observation_GUID",
              "Observation_Value_Numeric",
              "Observation_Value_CONCEPT",
              "Observation_Value_Name",
              "Unit_of_Measure",
              "c_Range"
            ]
          },
          {
            "name": "demographics",
            "columns": [
              "MRN",
              "Race",
              "Gender",
              "Maritial_Status",
              "Religion",
              "Occupation",
              "Address_State",
              "Blood_Type",
              "Rh_Factor"
            ]
          }
        ],
        "rootTable": "demographics",
        "rootColumn": "MRN",
        "follow": [
          "demographics__to__lab_event",
          "lab_event__to__lab_obs"
        ]
      }
    ]
  },
  "policies": {
    "stewards": [
      "CRISPI-TDR-training-admins@firecloud.org"
    ]
  },
  "tags": [
    "20240228_BTRIS_CRISPI_TDR_Workshop"
  ]
}

create_dataset_result = tdr_utils.wait_for_job(datasets_api.create_dataset(dataset=dataset_request))
# dataset = datasets_api.retrieve_dataset(create_dataset_result['id'])

Read in the dataset that was just created with full information

In [ ]:
dataset = datasets_api.retrieve_dataset(create_dataset_result['id'], include=["ACCESS_INFORMATION"])

# Ingest Data
Ingest tabular data into TDR.  These datasets do not contain files.

In [ ]:
demographics_records = [
        {
        "MRN": "1427981",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Other",
        "Maritial_Status_Concept": "C139560",
        "Religion": "Other",
        "Occupation": "NONE",
        "Language_Concept": "C2163463",
        "Phone_number": "195-9062",
        "Address_Line1": "63 Francis",
        "Addresss_City": "CHEVY CHASE",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "Blood_Type": "O ",
        "Rh_Factor": "P",
        "area_code": "301"
      }, {
        "MRN": "2167441",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Single",
        "Maritial_Status_Concept": "C113282",
        "Occupation": "CONSTRUCTION",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "195-8042",
        "Address_Line1": "64 First Street",
        "Addresss_City": "WASHINGTON",
        "Address_State": "DC",
        "Address_Country": "USA",
        "Birth_City": "OBERLIN, OH",
        "Data_Source": "C113092",
        "area_code": "540"
      }, {
        "MRN": "2477131",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Single",
        "Maritial_Status_Concept": "C113282",
        "Religion": "Christian Non-Denom",
        "Occupation": "RESEARCHER",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "196-2071",
        "Address_Line1": "60 Squash",
        "Addresss_City": "WASHINGTON",
        "Address_State": "DC",
        "Address_Country": "USA",
        "Birth_City": "MICHIGAN",
        "Data_Source": "C113092",
        "area_code": "703"
      }, {
        "MRN": "2075151",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Single",
        "Maritial_Status_Concept": "C113282",
        "Religion": "Presbyterian",
        "Occupation": "NONE",
        "Language_Concept": "C2163463",
        "Phone_number": "195-4051",
        "Address_Line1": "68 Pine",
        "Addresss_City": "ATLANTA",
        "Address_State": "GA",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "area_code": "404"
      }, {
        "MRN": "1420761",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Single",
        "Maritial_Status_Concept": "C113282",
        "Religion": "Christian",
        "Occupation": "STUDENT",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "198-1031",
        "Address_Line1": "41 Third",
        "Addresss_City": "SAN MARINO",
        "Address_State": "CA",
        "Address_Country": "USA",
        "Birth_City": "FAIRFAX, VA",
        "Data_Source": "C113092",
        "Blood_Type": "B",
        "Rh_Factor": "P",
        "area_code": "310"
      }, {
        "MRN": "3023921",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "Black/African Amer",
        "Ethnic_Group": "C16352",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Single",
        "Maritial_Status_Concept": "C113282",
        "Religion": "Lutheran",
        "Occupation": "NONE",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "195-5081",
        "Address_Line1": "67 Third",
        "Addresss_City": "SAN JUAN",
        "Address_State": "PR",
        "Address_Country": "USA",
        "Birth_City": "ANGUILLA",
        "Data_Source": "C113092",
        "Blood_Type": "AB",
        "Rh_Factor": "P",
        "area_code": "787"
      }, {
        "MRN": "2204831",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Baptist",
        "Occupation": "STUDENT",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "198-5051",
        "Address_Line1": "37 Spruce",
        "Addresss_City": "MANCHESTER",
        "Address_State": "OK",
        "Address_Country": "USA",
        "Birth_City": "PRATT, KANSAS",
        "Data_Source": "C113092",
        "Blood_Type": "O ",
        "Rh_Factor": "P",
        "area_code": "580"
      }, {
        "MRN": "921471",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Catholic",
        "Occupation": "DRAFTSMAN",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "196-3060",
        "Address_Line1": "59 Third",
        "Addresss_City": "TITUSVILLE",
        "Address_State": "FL",
        "Address_Country": "USA",
        "Birth_City": "WITCHITA",
        "Birth_State": "KS",
        "Birth_Country": "USA",
        "Data_Source": "C113092",
        "Blood_Type": "O ",
        "Rh_Factor": "P",
        "area_code": "321"
      }, {
        "MRN": "1614481",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Other",
        "Occupation": "NONE",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "194-8062",
        "Address_Line1": "74 Second",
        "Addresss_City": "POTOMAC",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "area_code": "301"
      }, {
        "MRN": "1753661",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Methodist",
        "Occupation": "NONE",
        "Language_Concept": "C2163463",
        "Phone_number": "196-2050",
        "Address_Line1": "60 Francis",
        "Addresss_City": "FREDERICKSBURG",
        "Address_State": "VA",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "Blood_Type": "B",
        "Rh_Factor": "P",
        "area_code": "540"
      }, {
        "MRN": "1694501",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C114517",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Methodist",
        "Occupation": "NONE",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "195-7020",
        "Address_Line1": "65 First Street",
        "Addresss_City": "ROCKVILLE",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "Blood_Type": "O ",
        "Rh_Factor": "P",
        "area_code": "240"
      }, {
        "MRN": "1952871",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Presbyterian",
        "Occupation": "NONE",
        "Language_Concept": "C2163463",
        "Phone_number": "194-2101",
        "Address_Line1": "79 Squash",
        "Addresss_City": "ROCKVILLE",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Data_Source": "C113092",
        "Blood_Type": "A",
        "Rh_Factor": "P",
        "area_code": "301"
      }, {
        "MRN": "1367981",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Male",
        "Gender_Concept": "C46109",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Catholic",
        "Occupation": "COMMERCIAL SALES",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "196-2092",
        "Address_Line1": "60 Green",
        "Addresss_City": "BRISTOL",
        "Address_State": "TN",
        "Address_Country": "USA",
        "Birth_City": "VA",
        "Data_Source": "C113092",
        "area_code": "423"
      }, {
        "MRN": "1929251",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "White",
        "Ethnic_Group": "C41261",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Baptist",
        "Occupation": "TAX DEPUTY",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "197-0073",
        "Address_Line1": "52 Berry",
        "Addresss_City": "ALDERSON",
        "Address_State": "WV",
        "Address_Country": "USA",
        "Birth_City": "WV",
        "Data_Source": "C113092",
        "Blood_Type": "B ",
        "Rh_Factor": "P",
        "area_code": "304"
      }, {
        "MRN": "1651281",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "Unknown",
        "Ethnic_Group": "C113269",
        "Ethnic_Group_Concept": "C17459",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "No Preference",
        "Occupation": "AIDE",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "197-5113",
        "Address_Line1": "46 Pine",
        "Addresss_City": "FREDERICK",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Birth_City": "CA",
        "Data_Source": "C113092",
        "Blood_Type": "B ",
        "Rh_Factor": "P",
        "area_code": "240"
      }, {
        "MRN": "1894981",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "Black/African Amer",
        "Ethnic_Group": "C16352",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Married",
        "Maritial_Status_Concept": "C51773",
        "Religion": "Catholic",
        "Occupation": "TUTOR",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "194-8041",
        "Address_Line1": "74 Spruce",
        "Addresss_City": "WASHINGTON",
        "Address_State": "DC",
        "Address_Country": "USA",
        "Birth_City": "WASHINGTON, DC",
        "Data_Source": "C113092",
        "Blood_Type": "O ",
        "Rh_Factor": "P",
        "area_code": "202"
      }, {
        "MRN": "2765211",
        "First_Name": "TEST",
        "Last_Name": " PATIENT",
        "Race": "Black/African Amer",
        "Ethnic_Group": "C16352",
        "Ethnic_Group_Concept": "C41222",
        "Gender": "Female",
        "Gender_Concept": "C46110",
        "Maritial_Status": "Divorced",
        "Maritial_Status_Concept": "C51776",
        "Religion": "Christian Non-Denom",
        "Occupation": "RELIEF COUNSELOR",
        "Language": "English",
        "Language_Concept": "C113896",
        "Phone_number": "195-3031",
        "Address_Line1": "69 Green",
        "Addresss_City": "SILVER SPRING",
        "Address_State": "MD",
        "Address_Country": "USA",
        "Birth_City": "AFRICA",
        "Data_Source": "C113092",
        "area_code": "301"
      }]

lab_event_records = [
        {
          "MRN": "1929251",
          "Event_GUID": "153853397",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1991-11-21 11:48:00",
          "Event_ID": "CH1461",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1420761",
          "Event_GUID": "145046851",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1983-01-03 10:30:00",
          "Event_ID": "CH4344",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1651281",
          "Event_GUID": "123743861",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1989-08-29 09:14:00",
          "Event_ID": "CH1611",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "3023921",
          "Event_GUID": "164614011",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1999-02-02 07:40:00",
          "Event_ID": "CH0643",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1952871",
          "Event_GUID": "136349159",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1992-03-26 08:24:00",
          "Event_ID": "CH1996",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1427981",
          "Event_GUID": "123747049",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1989-08-09 09:00:00",
          "Event_ID": "CH1318",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2204831",
          "Event_GUID": "171149340",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1992-03-12 04:46:00",
          "Event_ID": "CH4674",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1367981",
          "Event_GUID": "126304421",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1983-02-18 10:54:00",
          "Event_ID": "CH2496",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "921471",
          "Event_GUID": "136666726",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1992-02-10 11:02:00",
          "Event_ID": "CH8161",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2204831",
          "Event_GUID": "153313780",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1992-03-30 12:16:00",
          "Event_ID": "CH3856",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1694501",
          "Event_GUID": "146432084",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1984-12-05 08:26:00",
          "Event_ID": "CH9409",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2204831",
          "Event_GUID": "154833716",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1997-09-15 08:52:00",
          "Event_ID": "CH7305",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1753661",
          "Event_GUID": "141550005",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1994-04-14 09:48:00",
          "Event_ID": "CH6947",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1614481",
          "Event_GUID": "177293986",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1983-12-09 08:46:00",
          "Event_ID": "CH7991",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2167441",
          "Event_GUID": "162336636",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1995-02-23 10:44:00",
          "Event_ID": "CH6372",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2477131",
          "Event_GUID": "131189475",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1996-07-30 10:00:00",
          "Event_ID": "CH6601",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "1894981",
          "Event_GUID": "144810434",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1998-08-11 09:56:00",
          "Event_ID": "CH4783",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2765211",
          "Event_GUID": "155989201",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1998-02-19 11:58:00",
          "Event_ID": "CH9681",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "921471",
          "Event_GUID": "144512624",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1987-09-17 08:18:00",
          "Event_ID": "CH4570",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }, {
          "MRN": "2075151",
          "Event_GUID": "123338347",
          "Child_Flag": "false",
          "Event_Name": "GLUCOSE",
          "Event_Name_CONCEPT": "C2098502",
          "Primary_Date_Time": "1994-08-10 09:20:00",
          "Event_ID": "CH6150",
          "Sequence": "1.0",
          "Appl_Source_CD": "C140361",
          "Blinded_Flag": "false",
          "Date_created": "2010-01-22 22:18:22.840000",
          "Event_ID_1": "C119570"
        }]

lab_obs_records = [
        {
        "Observation_GUID": "617019732",
        "Event_GUID": "131189475",
        "MRN": "2477131",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1996-07-30 10:00:00",
        "Observation_Value_Numeric": "133",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "663513113",
        "Event_GUID": "177293986",
        "MRN": "1614481",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1983-12-09 08:46:00",
        "Observation_Value_Numeric": "88",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "605932352",
        "Event_GUID": "123743861",
        "MRN": "1651281",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1989-08-29 09:14:00",
        "Observation_Value_Numeric": "85",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "642740212",
        "Event_GUID": "155989201",
        "MRN": "2765211",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1998-02-19 11:58:00",
        "Observation_Value_Numeric": "99",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "616795808",
        "Event_GUID": "153853397",
        "MRN": "1929251",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1991-11-21 11:48:00",
        "Observation_Value_Numeric": "103",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "647503054",
        "Event_GUID": "144512624",
        "MRN": "921471",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1987-09-17 08:18:00",
        "Observation_Value_Numeric": "95",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "629321787",
        "Event_GUID": "153313780",
        "MRN": "2204831",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1992-03-30 12:16:00",
        "Observation_Value_Numeric": "98",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "636812777",
        "Event_GUID": "123338347",
        "MRN": "2075151",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1994-08-10 09:20:00",
        "Observation_Value_Numeric": "118",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "666759540",
        "Event_GUID": "162336636",
        "MRN": "2167441",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1995-02-23 10:44:00",
        "Observation_Value_Numeric": "61",
        "Observation_Value_CONCEPT": "C119022",
        "Observation_Value_Name": "L ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "615533186",
        "Event_GUID": "141550005",
        "MRN": "1753661",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1994-04-14 09:48:00",
        "Observation_Value_Numeric": "92",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "629321220",
        "Event_GUID": "171149340",
        "MRN": "2204831",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1992-03-12 04:46:00",
        "Observation_Value_Numeric": "201",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "604691520",
        "Event_GUID": "136349159",
        "MRN": "1952871",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1992-03-26 08:24:00",
        "Observation_Value_Numeric": "114",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "634887148",
        "Event_GUID": "126304421",
        "MRN": "1367981",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1983-02-18 10:54:00",
        "Observation_Value_Numeric": "81",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "640538848",
        "Event_GUID": "144810434",
        "MRN": "1894981",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1998-08-11 09:56:00",
        "Observation_Value_Numeric": "174",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "652179685",
        "Event_GUID": "123747049",
        "MRN": "1427981",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1989-08-09 09:00:00",
        "Observation_Value_Numeric": "138",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "649048851",
        "Event_GUID": "146432084",
        "MRN": "1694501",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1984-12-05 08:26:00",
        "Observation_Value_Numeric": "126",
        "Observation_Value_CONCEPT": "C119021",
        "Observation_Value_Name": "H ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "629325084",
        "Event_GUID": "154833716",
        "MRN": "2204831",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1997-09-15 08:52:00",
        "Observation_Value_Numeric": "95",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "647506617",
        "Event_GUID": "136666726",
        "MRN": "921471",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1992-02-10 11:02:00",
        "Observation_Value_Numeric": "82",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "645178594",
        "Event_GUID": "164614011",
        "MRN": "3023921",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1999-02-02 07:40:00",
        "Observation_Value_Numeric": "89",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }, {
        "Observation_GUID": "611802991",
        "Event_GUID": "145046851",
        "MRN": "1420761",
        "Child_Flag": "false",
        "Observation_Name": "GLUCOSE",
        "Observation_Name_CONCEPT": "C2098502",
        "Primary_Date_Time": "1983-01-03 10:30:00",
        "Observation_Value_Numeric": "80",
        "Observation_Value_CONCEPT": "C2097215",
        "Observation_Value_Name": "N ",
        "Sequence": "1.0",
        "Unit_of_Measure": "MG/DL",
        "Blinded_Flag": "false",
        "Appl_Source_CD": "C140361",
        "Date_Created": "2010-01-23 23:10:20.183000",
        "Domain_Concept": "C119570"
      }]

In [ ]:
# from tdr_utils import TdrUtils
demographics_ingest_request = {
  "format": "array",
  "ignore_unknown_values": True,
  "max_bad_records": 0,
  "table": "demographics",
  "records": demographics_records
}
demographics_result = tdr_utils.wait_for_job(datasets_api.ingest_dataset(dataset.id, ingest=demographics_ingest_request))



In [ ]:
lab_event_ingest_request = {
  "format": "array",
  "ignore_unknown_values": True,
  "max_bad_records": 0,
  "table": "lab_event",
  "records": lab_event_records
}
lab_event_result = tdr_utils.wait_for_job(datasets_api.ingest_dataset(dataset.id, ingest=lab_event_ingest_request))

In [ ]:
lab_obs_ingest_request = {
  "format": "array",
  "ignore_unknown_values": True,
  "max_bad_records": 0,
  "table": "lab_obs",
  "records": lab_obs_records
}
lab_obs_result = tdr_utils.wait_for_job(datasets_api.ingest_dataset(dataset.id, ingest=lab_obs_ingest_request))